# 如何构建多代理网络（功能性API）

!!!信息“先决条件”
本指南假设您熟悉以下内容：

- [多代理系统](../../concepts/multi_agent)
- [功能 API](../../concepts/function_api)
- [命令](../../concepts/low_level/#command)
- [LangGraph 术语表](../../concepts/low_level/)

在本操作指南中，我们将演示如何实现[多代理网络](../../concepts/multi_agent#network)架构，其中每个代理都可以与其他每个代理进行通信（多对多连接），并可以决定接下来调用哪个代理。我们将使用 LangGraph 的 [功能 API](../../concepts/function_api) — 各个代理将被定义为任务，代理切换将在主 [entrypoint()](/langgraphjs/reference/functions/langgraph.entrypoint-1.html) 中定义：
```ts
import { entrypoint, task } from "@langchain/langgraph";
import { createReactAgent } from "@langchain/langgraph/prebuilt";
import { tool } from "@langchain/core/tools";
import { z } from "zod";

// Define a tool to signal intent to hand off to a different agent
const transferToHotelAdvisor = tool(async () => {
  return "Successfully transferred to hotel advisor";
}, {
  name: "transferToHotelAdvisor",
  description: "Ask hotel advisor agent for help.",
  schema: z.object({}),
  returnDirect: true,
});

// define an agent
const travelAdvisorTools = [transferToHotelAdvisor, ...];
const travelAdvisor = createReactAgent({
  llm: model,
  tools: travelAdvisorTools,
});

// define a task that calls an agent
const callTravelAdvisor = task("callTravelAdvisor", async (messages: BaseMessage[]) => {
  const response = travelAdvisor.invoke({ messages });
  return response.messages;
});

const networkGraph = entrypoint(
  { name: "networkGraph" },
  async (messages: BaseMessageLike[]) => {
    let callActiveAgent = callTravelAdvisor;
    let agentMessages;
    while (true) {
      agentMessages = await callActiveAgent(messages);
      messages = addMessages(messages, agentMessages);
      callActiveAgent = getNextAgent(messages);
    }
    return messages;
  });
```

## 设置

!!!注意兼容性

本指南需要 `@langchain/langgraph>=0.2.42`。

首先，安装本示例所需的依赖项：
```bash
npm install @langchain/langgraph @langchain/anthropic @langchain/core zod
```
接下来，我们需要为 Anthropic 设置 API 密钥（我们将使用的 LLM）：
```typescript
process.env.ANTHROPIC_API_KEY = "YOUR_API_KEY";
```
!!!提示“为 LangGraph 开发设置 [LangSmith](https://smith.langchain.com)”

注册 LangSmith 以快速发现问题并提高 LangGraph 项目的性能。LangSmith 允许您使用跟踪数据来调试、测试和监控使用 LangGraph 构建的 LLM 应用程序 — 在[此处](https://docs.smith.langchain.com)了解有关如何开始的更多信息

## 旅行社示例

在此示例中，我们将建立一个可以相互通信的旅行助理代理团队。

我们将创建 2 个代理：

* `travelAdvisor`：可以帮助推荐旅行目的地。可以向`hotelAdvisor`寻求帮助。
* `hotelAdvisor`：可以帮助推荐酒店。可以向`travelAdvisor`寻求帮助。

这是一个完全连接的网络 - 每个代理都可以与任何其他代理通信。

首先，让我们创建代理将使用的一些工具：

In [1]:
import { tool } from "@langchain/core/tools";
import { z } from "zod";

// 获取旅行建议的工具
const getTravelRecommendations = tool(async () => {
  const destinations = ["aruba", "turks and caicos"];
  return destinations[Math.floor(Math.random() * destinations.length)];
}, {
  name: "getTravelRecommendations",
  description: "Get recommendation for travel destinations",
  schema: z.object({}),
});

// 获取酒店推荐的工具
const getHotelRecommendations = tool(async (input: { location: "aruba" | "turks and caicos" }) => {
  const recommendations = {
    "aruba": [
      "The Ritz-Carlton, Aruba (Palm Beach)",
      "Bucuti & Tara Beach Resort (Eagle Beach)"
    ],
    "turks and caicos": ["Grace Bay Club", "COMO Parrot Cay"]
  };
  return recommendations[input.location];
}, {
  name: "getHotelRecommendations",
  description: "Get hotel recommendations for a given destination.",
  schema: z.object({
    location: z.enum(["aruba", "turks and caicos"])
  }),
});

// 定义一个工具来发出移交给不同代理的意图
// 注意：这不是使用 Command(goto) 语法导航到不同的代理：
// 下面的“workflow()”显式处理切换
const transferToHotelAdvisor = tool(async () => {
  return "Successfully transferred to hotel advisor";
}, {
  name: "transferToHotelAdvisor",
  description: "Ask hotel advisor agent for help.",
  schema: z.object({}),
  // 提示我们的代理实施应该停止
  // 调用此工具后立即
  returnDirect: true,
}); 

const transferToTravelAdvisor = tool(async () => {
  return "Successfully transferred to travel advisor";
}, {
  name: "transferToTravelAdvisor", 
  description: "Ask travel advisor agent for help.",
  schema: z.object({}),
  // 提示我们的代理实施应该停止
  // 调用此工具后立即
  returnDirect: true,
});

!!!注意“传输工具”

您可能已经注意到，我们在传输工具中使用了 `tool(... { returnDirect: true })`。这样做是为了在调用这些工具后，各个代理（例如，`travelAdvisor`）可以尽早退出 ReAct 循环，而无需最后一次调用模型来处理工具调用的结果。这是期望的行为，因为我们希望检测代理何时调用此工具并将控制权“立即”移交给不同的代理。

**注意**：这意味着与预构建的 [`createReactAgent`](/langgraphjs/reference/functions/langgraph_prebuilt.createReactAgent.html) 一起使用 - 如果您正在构建自定义代理，请确保手动添加逻辑以处理标记为 `returnDirect` 的工具的提前退出。

现在让我们定义代理任务并将它们组合成单个多代理网络工作流程：

In [2]:
import {
  AIMessage,
  type BaseMessageLike
} from "@langchain/core/messages";
import { ChatAnthropic } from "@langchain/anthropic";
import { createReactAgent } from "@langchain/langgraph/prebuilt";
import {
  addMessages,
  entrypoint,
  task,
} from "@langchain/langgraph";

const model = new ChatAnthropic({
  model: "claude-3-5-sonnet-latest",
});

const travelAdvisorTools = [
  getTravelRecommendations,
  transferToHotelAdvisor,
];

// 定义旅行顾问 ReAct 代理
const travelAdvisor = createReactAgent({
  llm: model,
  tools: travelAdvisorTools,
  stateModifier: [
    "You are a general travel expert that can recommend travel destinations (e.g. countries, cities, etc).",
    "If you need hotel recommendations, ask 'hotel_advisor' for help.",
    "You MUST include human-readable response before transferring to another agent.",
  ].join(" "),
});

// 您还可以添加其他逻辑，例如更改代理的输入/代理的输出等。
// 注意：我们正在使用状态中消息的完整历史记录调用 ReAct 代理
const callTravelAdvisor = task("callTravelAdvisor", async (messages: BaseMessageLike[]) => {
  const response = await travelAdvisor.invoke({ messages });
  return response.messages;
});

const hotelAdvisorTools = [
  getHotelRecommendations,
  transferToTravelAdvisor,
];

// 定义酒店顾问 ReAct 代理
const hotelAdvisor = createReactAgent({
  llm: model,
  tools: hotelAdvisorTools,
  stateModifier: [
    "You are a hotel expert that can provide hotel recommendations for a given destination.",
    "If you need help picking travel destinations, ask 'travel_advisor' for help.",
    "You MUST include a human-readable response before transferring to another agent."
  ].join(" "),
});

// 添加酒店顾问任务
const callHotelAdvisor = task("callHotelAdvisor", async (messages: BaseMessageLike[]) => {
  const response = await hotelAdvisor.invoke({ messages });
  return response.messages;
});

const networkGraph = entrypoint(
  "networkGraph",
  async (messages: BaseMessageLike[]) => {
    // 将输入转换为 LangChain 消息作为副作用
    let currentMessages = addMessages([], messages);

    let callActiveAgent = callTravelAdvisor;
    while (true) {
      const agentMessages = await callActiveAgent(currentMessages);
      currentMessages = addMessages(currentMessages, agentMessages);
      
      // 查找最后一条 AI 消息
      // 如果调用其中一个切换工具，则返回最后一条消息
      // 代理将是一个 ToolMessage 因为我们将它们设置为
      // “returnDirect：true”。这意味着最后一条 AIMessage 将
      // 有工具调用。
      // 否则，最后返回的消息将是一条 AIMessage
      // 没有工具调用，这意味着我们已经准备好接受新的输入。
      const aiMsg = [...agentMessages].reverse()
        .find((m): m is AIMessage => m.getType() === "ai");
        
      // 如果没有工具调用，我们就完成了
      if (!aiMsg?.tool_calls?.length) {
        break;
      }

      // 获取最后一次工具调用并确定下一个代理
      const toolCall = aiMsg.tool_calls.at(-1)!;
      if (toolCall.name === "transferToTravelAdvisor") {
        callActiveAgent = callTravelAdvisor;
      } else if (toolCall.name === "transferToHotelAdvisor") {
        callActiveAgent = callHotelAdvisor;
      } else {
        throw new Error(`Expected transfer tool, got '${toolCall.name}'`);
      }
    }

    return messages;
  });

最后，让我们定义一个助手来呈现代理输出：

In [4]:
const prettyPrintMessages = (update: Record<string, any>) => {
  // 使用命名空间处理元组情况
  if (Array.isArray(update)) {
    const [ns, updateData] = update;
    // 在打印输出中跳过父图更新
    if (ns.length === 0) {
      return;
    }

    const graphId = ns[ns.length - 1].split(":")[0];
    console.log(`Update from subgraph ${graphId}:\n`);
    update = updateData;
  }

  if (update.__metadata__?.cached) {
    return;
  }
  // 打印每个节点的更新
  for (const [nodeName, updateValue] of Object.entries(update)) {
    console.log(`Update from node ${nodeName}:\n`);

    const coercedMessages = addMessages([], updateValue.messages);
    for (const message of coercedMessages) {
      const textContent = typeof message.content === "string"
        ? message.content
        : JSON.stringify(message.content);
      // 根据角色打印消息内容
      if (message.getType() === "ai") {
        console.log("=".repeat(33) + " Assistant Message " + "=".repeat(33));
        console.log(textContent);
        console.log();
      } else if (message.getType() === "human") {
        console.log("=".repeat(33) + " Human Message " + "=".repeat(33));
        console.log(textContent);
        console.log();
      } else if (message.getType() === "tool") {
        console.log("=".repeat(33) + " Tool Message " + "=".repeat(33));
        console.log(textContent);
        console.log();
      }
    }
    console.log("\n");
  }
};

让我们测试一下：

In [5]:
const stream = await networkGraph.stream([{
  role: "user",
  content: "i wanna go somewhere warm in the caribbean. pick one destination and give me hotel recommendations"
}], { subgraphs: true })

for await (const chunk of stream) {
  prettyPrintMessages(chunk);
}

Update from subgraph callTravelAdvisor:

Update from node agent:

================================= Assistant Message =================================
[{"type":"text","text":"I'll help you find a warm Caribbean destination and then get specific hotel recommendations for you.\n\nLet me first get some destination recommendations for the Caribbean region."},{"type":"tool_use","id":"toolu_019fN1etkqtCSausSv8XufhL","name":"getTravelRecommendations","input":{}}]



Update from subgraph callTravelAdvisor:

Update from node tools:

================================= Tool Message =================================
turks and caicos



Update from subgraph callTravelAdvisor:

Update from node agent:

================================= Assistant Message =================================
[{"type":"text","text":"Great! I recommend Turks and Caicos for your Caribbean getaway. This beautiful British Overseas Territory is known for its stunning white-sand beaches, crystal-clear turquoise waters, and perf

瞧 - `travelAdvisor` 选择一个目的地，然后决定致电 `hotelAdvisor` 获取更多信息！